Connect PIC

In [1]:
%reload_ext autoreload
%autoreload

import dut_func as dut
import serial
import time
import struct
import numpy as np
# from bitarray import bitarray
from bitstring import BitArray

from misc import *

# dut.drv.serUSB = 'COM19'
drv = dut.drv

ser = serial.Serial('COM5', 9600, timeout=1)
dut.connect(ser)

Disconnect PIC

In [10]:
dut.disconnect()

Serial port disconnected.


Run this before bringing up 5V and +-20V supplies but after 6V supply

In [2]:
dut.pads_defaults()
dut.vrefs_off()
drv.gpio_pin_reset(*PIC_PINS['NRESET_FULL_CHIP'])
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
drv.gpio_pin_reset(*PIC_PINS['PWR_GOOD'])

DAC initialized to a span from -5 V to 5 V


Once DUT power brought up, bring up DAC voltages

In [32]:
dut.vrefs_defaults()

In [6]:
dut.dac_set('P_TVDD', 5)

In [97]:
dut.dac_set('PLANE_VPP', 0.02)

Now assert power good

In [31]:
drv.gpio_pin_set(*PIC_PINS['PWR_GOOD'])

In [3]:
dut.power_on()

In [16]:
dut.py_logic_analyzer()

1 	PWR_GOOD
0 	NRESET_FULL_CHIP
0 	NRESET_DPE_ENGINE
0 	ARRAY_EN<0>
0 	ARRAY_EN<1>
0 	ARRAY_EN<2>
0 	NFORCE_SAFE0
0 	NFORCE_SAFE1
0 	NFORCE_SAFE2
0 	ADC_SEL_EXT
0 	DPE_EXT_OVERRIDE_EN
0 	DPE_EXT_SH
1 	ADC_DONE
0 	ADC_FIFO_ADVANCE
0 	ADC_FIFO_EN<0>
0 	ADC_FIFO_EN<1>
0 	ADC_FIFO_EN<2>
0 	ADC_FIFO_EN<3>


Bring full chip out of reset

In [9]:
portName, pinPos = PIC_PINS['NRESET_FULL_CHIP']
drv.gpio_pin_reset(portName, pinPos)
time.sleep(0.1) # want to delay 1us
drv.gpio_pin_set(portName, pinPos)

Scan in control values. Must do this before setting NRESET_DPE_ENGINE

In [10]:
# Control block scan chain for power up LEAVE ALONE
data = bytes([0b000_10000, 0b0000_0010 , 0b0000_1100,0b0001_0000, 0b0010_0000, 0b00000001, 0b00000010])
drv.spi_serial_write(2, data) # addr 1 here means SERIAL_CHAIN_SEL0 is 1 and _SEL1 is 0

Start the clocks

In [33]:
drv.clk_start('CK_ARRAY')
drv.clk_start('ADC_CK')

Calibrate the ADCs

In [8]:
dut.dac_set('P_ADC_EXT_TEST_IN', 0.658)

In [9]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
time.sleep(1e-6) # want to delay 1us
drv.gpio_pin_set(*PIC_PINS['NRESET_DPE_ENGINE'])

In [34]:
drv.gpio_pin_set(*PIC_PINS['NRESET_FULL_CHIP'])

In [6]:
dut.pads_defaults()

In [7]:
dut.pads_defaults()

In [8]:
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE0'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE1'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE2'])

In [12]:
drv.gpio_pin_set(*PIC_PINS['ADC_SEL_EXTERNAL'])

In [13]:
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_OVERRIDE_EN'])

In [14]:
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_SH'])

Check for an ADC_DONE signal

In [9]:
dut.py_logic_analyzer()

1 	PWR_GOOD
1 	NRESET_FULL_CHIP
1 	NRESET_DPE_ENGINE
0 	ARRAY_EN<0>
0 	ARRAY_EN<1>
0 	ARRAY_EN<2>
1 	NFORCE_SAFE0
1 	NFORCE_SAFE1
1 	NFORCE_SAFE2
0 	ADC_SEL_EXT
0 	DPE_EXT_OVERRIDE_EN
0 	DPE_EXT_SH
0 	ADC_DONE
0 	ADC_FIFO_ADVANCE
0 	ADC_FIFO_EN<0>
0 	ADC_FIFO_EN<1>
0 	ADC_FIFO_EN<2>
0 	ADC_FIFO_EN<3>


In [16]:
drv.gpio_pin_reset(*PIC_PINS['DPE_EXT_SH'])

In [35]:
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

Dump all of the FIFOs

In [36]:
def adc2volt(data):
    V_HI = 4.0
    V_LO = 0.5
    value = data & 0x3ff
    voltage = (V_HI-V_LO) * value / 1023 + V_LO
    return voltage

drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()


0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V
0000000000010	0.507 V



In [6]:
dut.pads_defaults()

In [37]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])

Internal Read without AGC

In [67]:
portName, pinPos = PIC_PINS['NRESET_FULL_CHIP']
drv.gpio_pin_reset(portName, pinPos)
time.sleep(0.1) # want to delay 1us
drv.gpio_pin_set(portName, pinPos)

In [68]:
# Control block scan chain for power up LEAVE ALONE
data = bytes([0b0001_0000, 0b0000_0010 , 0b0000_1100,0b0001_0000, 0b0010_0000, 0b0000_0001, 0b0000_0010])
drv.spi_serial_write(2, data) # addr 1 here means SERIAL_CHAIN_SEL0 is 1 and _SEL1 is 0

In [20]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
dut.pads_defaults()
drv.gpio_pin_set(*PIC_PINS['NRESET_DPE_ENGINE'])
dut.pads_defaults()

Scan in control and TIA values

In [21]:
dut.py_logic_analyzer()

1 	PWR_GOOD
1 	NRESET_FULL_CHIP
1 	NRESET_DPE_ENGINE
0 	ARRAY_EN<0>
0 	ARRAY_EN<1>
0 	ARRAY_EN<2>
0 	NFORCE_SAFE0
0 	NFORCE_SAFE1
0 	NFORCE_SAFE2
0 	ADC_SEL_EXT
0 	DPE_EXT_OVERRIDE_EN
0 	DPE_EXT_SH
1 	ADC_DONE
0 	ADC_FIFO_ADVANCE
0 	ADC_FIFO_EN<0>
0 	ADC_FIFO_EN<1>
0 	ADC_FIFO_EN<2>
0 	ADC_FIFO_EN<3>


In [69]:
# TIA SCAN IN OP AMPS ENABLED, No COMPENSATION, 30k gain
data = bytes([0b1100_0100, 0b0011_0001, 0b0000_1100, 0b0100_0011, 0b0001_0000]*24)
#1kohm	5kohm	30kohm	200kohm	1Mohm	cm	C<1>	C<0>	TIA en	S/H en
#data = BitArray('0b1100000001'*96).bytes
#data = BitArray('0b11000000011100000010110000000111000001001100000010110000100011000000011100010000'*12).bytes

print(data)
drv.spi_serial_write(0, data)
# TIA SCAN IN OP AMPS ENABLED, No COMPENSATION, 30k gain
#data = bytes([0b1100_0001, 0b0011_0000, 0b0100_1100, 0b0001_0011, 0b0000_0100]*24)
print(data)
drv.spi_serial_write(1, data)

time.sleep(1e-6)
drv.gpio_pin_set(*PIC_PINS['UPDATE_TIA_CONF'])
time.sleep(1e-6)
drv.gpio_pin_reset(*PIC_PINS['UPDATE_TIA_CONF'])

b'\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10'
b'\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10\xc41\x0cC\x10'


In [70]:
roc = [0 ,0, 0]
# data= 0b1111111111111111
data= 0xFFFF
array= [0, 1, 2]
N = 3
for a in range(0, N-1):
    drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])
    if roc[a] == 1:
        drv.gpio_pin_set(*PIC_PINS['COL_ROW_SEL'])
    elif roc[a] == 0:
        drv.gpio_pin_reset(*PIC_PINS['COL_ROW_SEL'])
    else:
        print('error, roc = 0 or 1')
        
    drv.gpio_row_col_bank_write(0b1000)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0100)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0010)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0001)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_pin_reset(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])

In [71]:
roc = [1 ,1, 1]
# data= 0b1111111111111111
data= 0xFFFF
array= [0, 1, 2]
N = 3
for a in range(0, N-1):
    drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])
    if roc[a] == 1:
        drv.gpio_pin_set(*PIC_PINS['COL_ROW_SEL'])
    elif roc[a] == 0:
        drv.gpio_pin_reset(*PIC_PINS['COL_ROW_SEL'])
    else:
        print('error, roc = 0 or 1')
        
    drv.gpio_row_col_bank_write(0b1000)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0100)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0010)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0001)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_pin_reset(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])

In [72]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
dut.pads_defaults()
drv.gpio_pin_set(*PIC_PINS['NRESET_DPE_ENGINE'])

In [37]:
dut.py_logic_analyzer()

1 	PWR_GOOD
1 	NRESET_FULL_CHIP
1 	NRESET_DPE_ENGINE
0 	ARRAY_EN<0>
0 	ARRAY_EN<1>
0 	ARRAY_EN<2>
0 	NFORCE_SAFE0
0 	NFORCE_SAFE1
0 	NFORCE_SAFE2
0 	ADC_SEL_EXT
0 	DPE_EXT_OVERRIDE_EN
0 	DPE_EXT_SH
1 	ADC_DONE
0 	ADC_FIFO_ADVANCE
0 	ADC_FIFO_EN<0>
0 	ADC_FIFO_EN<1>
0 	ADC_FIFO_EN<2>
0 	ADC_FIFO_EN<3>


In [38]:
drv.gpio_pin_set(*PIC_PINS['DPE_INTERNAL_EN'])
drv.gpio_pin_set(*PIC_PINS['AGC_INTERNAL_EN'])

In [39]:
drv.gpio_pin_set(*PIC_PINS['READ_BIT'])
drv.gpio_pin_set(*PIC_PINS['READ_DPE'])

In [40]:
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE0'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE1'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE2'])

In [41]:
drv.gpio_pin_set(*PIC_PINS['CONNECT_TIA'])

In [42]:
drv.gpio_pin_set(*PIC_PINS['CONNECT_COLUMN_T'])

In [43]:
#drv.gpio_pin_set(*PIC_PINS['DPE_PULSE'])
drv.gpio_pin_set(*PIC_PINS['AGC_PULSE'])

In [44]:
dut.py_logic_analyzer()

1 	PWR_GOOD
1 	NRESET_FULL_CHIP
1 	NRESET_DPE_ENGINE
0 	ARRAY_EN<0>
0 	ARRAY_EN<1>
0 	ARRAY_EN<2>
1 	NFORCE_SAFE0
1 	NFORCE_SAFE1
1 	NFORCE_SAFE2
0 	ADC_SEL_EXT
0 	DPE_EXT_OVERRIDE_EN
0 	DPE_EXT_SH
1 	ADC_DONE
0 	ADC_FIFO_ADVANCE
0 	ADC_FIFO_EN<0>
0 	ADC_FIFO_EN<1>
0 	ADC_FIFO_EN<2>
0 	ADC_FIFO_EN<3>


In [45]:
dut.pads_defaults()

In [46]:
def adc2volt(data):
    V_HI = 4.0
    V_LO = 0.5
    value = data & 0x3ff
    voltage = (V_HI-V_LO) * value / 1023 + V_LO
    return voltage

In [202]:
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V

0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V

0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
0111111111111	4.000 V
01111111

Internal Read with AGC

In [39]:
portName, pinPos = PIC_PINS['NRESET_FULL_CHIP']
drv.gpio_pin_reset(portName, pinPos)
time.sleep(0.1) # want to delay 1us
drv.gpio_pin_set(portName, pinPos)

In [40]:
# Control block scan chain for power up LEAVE ALONE
data = bytes([0b0001_0000, 0b0000_0010 , 0b0000_1100,0b0001_0000, 0b0010_0000, 0b0000_0001, 0b0000_0010])
drv.spi_serial_write(2, data) # addr 1 here means SERIAL_CHAIN_SEL0 is 1 and _SEL1 is 0

In [41]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
dut.pads_defaults()
drv.gpio_pin_set(*PIC_PINS['NRESET_DPE_ENGINE'])
dut.pads_defaults()

In [15]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])

Scan in control and TIA values

In [16]:
dut.py_logic_analyzer()

1 	PWR_GOOD
1 	NRESET_FULL_CHIP
0 	NRESET_DPE_ENGINE
0 	ARRAY_EN<0>
0 	ARRAY_EN<1>
0 	ARRAY_EN<2>
0 	NFORCE_SAFE0
0 	NFORCE_SAFE1
0 	NFORCE_SAFE2
0 	ADC_SEL_EXT
0 	DPE_EXT_OVERRIDE_EN
0 	DPE_EXT_SH
1 	ADC_DONE
0 	ADC_FIFO_ADVANCE
0 	ADC_FIFO_EN<0>
0 	ADC_FIFO_EN<1>
0 	ADC_FIFO_EN<2>
0 	ADC_FIFO_EN<3>


In [42]:
# TIA SCAN IN OP AMPS ENABLED, No COMPENSATION, 30k gain
#data = bytes([0b1100_0100, 0b0011_0001, 0b0000_1100, 0b0100_0011, 0b0001_0000]*24)
#1kohm	5kohm	30kohm	200kohm	1Mohm	cm	C<1>	C<0>	TIA en	S/H en
#data = BitArray('0b1100010000'*96).bytes
data = BitArray('0b11000000011100000010110000000111000001001100000010110000100011000000011100010000'*12).bytes

print(data)
drv.spi_serial_write(0, data)
# TIA SCAN IN OP AMPS ENABLED, No COMPENSATION, 30k gain
#data = bytes([0b1100_0001, 0b0011_0000, 0b0100_1100, 0b0001_0011, 0b0000_0100]*24)
print(data)
drv.spi_serial_write(1, data)

time.sleep(1e-6)
drv.gpio_pin_set(*PIC_PINS['UPDATE_TIA_CONF'])
time.sleep(1e-6)
drv.gpio_pin_reset(*PIC_PINS['UPDATE_TIA_CONF'])

b'\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10'
b'\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10\xc0p,\x07\x04\xc0\xb0\x8c\x07\x10'


Load vectors to set all rows and columns to 1

In [43]:
roc = [0 ,0, 0]
# data= 0b1111111111111111
data= 0xFFFF
array= [0, 1, 2]
N = 3
for a in range(0, N-1):
    drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])
    if roc[a] == 1:
        drv.gpio_pin_set(*PIC_PINS['COL_ROW_SEL'])
    elif roc[a] == 0:
        drv.gpio_pin_reset(*PIC_PINS['COL_ROW_SEL'])
    else:
        print('error, roc = 0 or 1')
        
    drv.gpio_row_col_bank_write(0b1000)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0100)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0010)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0001)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_pin_reset(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])

In [44]:
roc = [1 ,1, 1]
# data= 0b1111111111111111
data= 0xFFFF
array= [0, 1, 2]
N = 3
for a in range(0, N-1):
    drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])
    if roc[a] == 1:
        drv.gpio_pin_set(*PIC_PINS['COL_ROW_SEL'])
    elif roc[a] == 0:
        drv.gpio_pin_reset(*PIC_PINS['COL_ROW_SEL'])
    else:
        print('error, roc = 0 or 1')
        
    drv.gpio_row_col_bank_write(0b1000)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0100)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0010)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0001)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_pin_reset(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])

In [56]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
dut.pads_defaults()
drv.gpio_pin_set(*PIC_PINS['NRESET_DPE_ENGINE'])

In [57]:
dut.pads_defaults()

In [22]:
dut.py_logic_analyzer()

1 	PWR_GOOD
1 	NRESET_FULL_CHIP
1 	NRESET_DPE_ENGINE
0 	ARRAY_EN<0>
0 	ARRAY_EN<1>
0 	ARRAY_EN<2>
0 	NFORCE_SAFE0
0 	NFORCE_SAFE1
0 	NFORCE_SAFE2
0 	ADC_SEL_EXT
0 	DPE_EXT_OVERRIDE_EN
0 	DPE_EXT_SH
1 	ADC_DONE
0 	ADC_FIFO_ADVANCE
0 	ADC_FIFO_EN<0>
0 	ADC_FIFO_EN<1>
0 	ADC_FIFO_EN<2>
0 	ADC_FIFO_EN<3>


In [58]:
drv.gpio_pin_set(*PIC_PINS['DPE_INTERNAL_EN'])
#drv.gpio_pin_set(*PIC_PINS['AGC_INTERNAL_EN'])

In [59]:
drv.gpio_pin_set(*PIC_PINS['READ_BIT'])
drv.gpio_pin_set(*PIC_PINS['READ_DPE'])

In [60]:
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE0'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE1'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE2'])

In [61]:
drv.gpio_pin_set(*PIC_PINS['CONNECT_TIA'])

In [62]:
drv.gpio_pin_set(*PIC_PINS['CONNECT_COLUMN_T'])

In [63]:
drv.gpio_pin_set(*PIC_PINS['DPE_PULSE'])
#drv.gpio_pin_set(*PIC_PINS['AGC_PULSE'])

In [64]:
dut.py_logic_analyzer()

1 	PWR_GOOD
1 	NRESET_FULL_CHIP
1 	NRESET_DPE_ENGINE
0 	ARRAY_EN<0>
0 	ARRAY_EN<1>
0 	ARRAY_EN<2>
1 	NFORCE_SAFE0
1 	NFORCE_SAFE1
1 	NFORCE_SAFE2
0 	ADC_SEL_EXT
0 	DPE_EXT_OVERRIDE_EN
0 	DPE_EXT_SH
1 	ADC_DONE
0 	ADC_FIFO_ADVANCE
0 	ADC_FIFO_EN<0>
0 	ADC_FIFO_EN<1>
0 	ADC_FIFO_EN<2>
0 	ADC_FIFO_EN<3>


In [53]:
dut.pads_defaults()

In [54]:
def adc2volt(data):
    V_HI = 4.0
    V_LO = 0.5
    value = data & 0x3ff
    voltage = (V_HI-V_LO) * value / 1023 + V_LO
    return voltage

In [65]:
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])



In [77]:
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])

In [55]:
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

0000000000010	0.507 V
0010000000010	0.507 V
0010000000010	0.507 V
0100000000010	0.507 V
0110000000010	0.507 V
0110011100010	1.273 V
0110000000010	0.507 V
0000000000010	0.507 V
0001110111111	3.781 V
0000000000010	0.507 V
0110010100010	1.054 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V

0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0100000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0000000100010	0.616 V
0001111111111	4.000 V
0000000000010	0.507 V
0110010000010	0.945 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V

0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0100000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0000001100010	0.835 V
0000001000010	0.726 V
0000000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
01100000

In [ ]:
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

TIA CALIBRATION

In [58]:
# Control block scan chain for power up LEAVE ALONE
data = bytes([0b0001_0000, 0b0000_0010 , 0b0000_1100,0b0001_0000, 0b0010_0000, 0b0000_0001, 0b0000_0010])
drv.spi_serial_write(2, data) # addr 1 here means SERIAL_CHAIN_SEL0 is 1 and _SEL1 is 0

In [71]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
dut.pads_defaults()
drv.gpio_pin_set(*PIC_PINS['NRESET_DPE_ENGINE'])

In [48]:
# TIA SCAN IN OP AMPS ENABLED, No COMPENSATION, 30k gain
data = bytes([0b1100_0001, 0b0011_0000, 0b0100_1100, 0b0001_0011, 0b0000_0100]*24)
#1kohm	5kohm	30kohm	200kohm	1Mohm	cm	C<1>	C<0>	TIA en	S/H en
#data = BitArray('0b1100000001'*96).bytes
#data = BitArray('0b11000000011100000010110000000111000001001100000010110000100011000000011100010000'*12).bytes

#print(data)
drv.spi_serial_write(0, data)
# TIA SCAN IN OP AMPS ENABLED, No COMPENSATION, 30k gain
#data = bytes([0b1100_0001, 0b0011_0000, 0b0100_1100, 0b0001_0011, 0b0000_0100]*24)
#print(data)
drv.spi_serial_write(1, data)

time.sleep(1e-6)
drv.gpio_pin_set(*PIC_PINS['UPDATE_TIA_CONF'])
time.sleep(1e-6)
drv.gpio_pin_reset(*PIC_PINS['UPDATE_TIA_CONF'])

In [95]:
roc = [0 ,0, 0]
# data= 0b1111111111111111
data= 0x0000
array= [0, 1, 2]
N = 3
for a in range(0, N-1):
    drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])
    if roc[a] == 1:
        drv.gpio_pin_set(*PIC_PINS['COL_ROW_SEL'])
    elif roc[a] == 0:
        drv.gpio_pin_reset(*PIC_PINS['COL_ROW_SEL'])
    else:
        print('error, roc = 0 or 1')
        
    drv.gpio_row_col_bank_write(0b1000)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0100)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0010)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0001)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_pin_reset(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])

In [96]:
roc = [1 ,1, 1]
# data= 0b1111111111111111
data= 0x0000
array= [0, 1, 2]
N = 3
for a in range(0, N-1):
    drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])
    if roc[a] == 1:
        drv.gpio_pin_set(*PIC_PINS['COL_ROW_SEL'])
    elif roc[a] == 0:
        drv.gpio_pin_reset(*PIC_PINS['COL_ROW_SEL'])
    else:
        print('error, roc = 0 or 1')
        
    drv.gpio_row_col_bank_write(0b1000)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0100)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0010)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_row_col_bank_write(0b0001)
    drv.gpio_row_col_data_write(data)
    time.sleep(1e-7)
    drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
    time.sleep(1e-7)
    drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

    drv.gpio_pin_reset(*PIC_PINS['ARRAY_EN<%d>' %(array[a])])

In [63]:
# LOAD ONE COLUMN ON ONE ARRAY (NOTE: THIS ASSUMES YOU"VE PREVIOUSLY LOADED ALL ZEROS)

drv.gpio_pin_reset(*PIC_PINS['ARRAY_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ARRAY_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ARRAY_EN<2>'])

drv.gpio_pin_set(*PIC_PINS['COL_ROW_SEL'])

drv.gpio_row_col_bank_write(0b0001)
drv.gpio_row_col_data_write(0b1000_0000_0000_0000)
time.sleep(1e-5)
drv.gpio_pin_set(*PIC_PINS['LATCH_CLK_DATA'])
time.sleep(1e-5)
drv.gpio_pin_reset(*PIC_PINS['LATCH_CLK_DATA'])

In [71]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
dut.pads_defaults()
drv.gpio_pin_set(*PIC_PINS['NRESET_DPE_ENGINE'])

In [123]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
dut.pads_defaults()
drv.gpio_pin_set(*PIC_PINS['NRESET_DPE_ENGINE'])

In [124]:
drv.gpio_pin_set(*PIC_PINS['WRITE_SEL_EXT'])
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_OVERRIDE_EN'])                 
time.sleep(5e-6)   
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE2'])
drv.gpio_pin_reset(*PIC_PINS['NFORCE_SAFE1'])
drv.gpio_pin_reset(*PIC_PINS['NFORCE_SAFE0'])
time.sleep(5e-6)        # delay(t_cal_start), min = 2TCK
drv.gpio_pin_set(*PIC_PINS['COL_WRITE_CONNECT'])
time.sleep(5e-6)        # delay(t_opamp), min = 500ns
drv.gpio_pin_set(*PIC_PINS['CONNECT_TIA'])
#drv.gpio_pin_set(*PIC_PINS['DPE_EXT_SH'])

In [125]:
# Before setting this, we're using an external current source to apply to the board!
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_SH'])
# Then see that ADC_DONE goes high and strobe out the FIFO

In [126]:
dut.py_logic_analyzer()

PWR_GOOD		 1
NRESET_FULL_CHIP		 1
NRESET_DPE_ENGINE		 1
ARRAY_EN<0>		 0
ARRAY_EN<1>		 0
ARRAY_EN<2>		 0
NFORCE_SAFE0		 0
NFORCE_SAFE1		 0
NFORCE_SAFE2		 1
ADC_SEL_EXT		 0
DPE_EXT_OVERRIDE_EN		 1
DPE_EXT_SH		 1
ADC_DONE<0>		 1
ADC_FIFO_ADVANCE		 0
ADC_FIFO_EN<0>		 0
ADC_FIFO_EN<1>		 0
ADC_FIFO_EN<2>		 0
ADC_FIFO_EN<3>		 0


In [127]:
dut.pads_defaults()

In [128]:
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

0100000000010	0.507 V
0100000000010	0.507 V
0100000000010	0.507 V
0100000000010	0.507 V
0100000000010	0.507 V
0100000000010	0.507 V
0100000000010	0.507 V
0100000000010	0.507 V
0100000000010	0.507 V
0100000000010	0.507 V
0100000000010	0.507 V
0100000000010	0.507 V
0100000000010	0.507 V
0100000000010	0.507 V
0100000000010	0.507 V
0100000000010	0.507 V



EXTERNAL READ

In [121]:
drv.gpio_pin_reset(*PIC_PINS['NRESET_DPE_ENGINE'])
dut.pads_defaults()
drv.gpio_pin_set(*PIC_PINS['NRESET_DPE_ENGINE'])

In [122]:
# TIA SCAN IN OP AMPS ENABLED, No COMPENSATION, 30k gain
#data = bytes([0b1100_0001, 0b0011_0000, 0b0100_1100, 0b0001_0011, 0b0000_0100]*24)
#1kohm	5kohm	30kohm	200kohm	1Mohm	cm	C<1>	C<0>	TIA en	S/H en
data = BitArray('0b1100000001'*96).bytes
#data = BitArray('0b11000000011100000010110000000111000001001100000010110000100011000000011100010000'*12).bytes

#print(data)
drv.spi_serial_write(0, data)
# TIA SCAN IN OP AMPS ENABLED, No COMPENSATION, 30k gain
#data = bytes([0b1100_0001, 0b0011_0000, 0b0100_1100, 0b0001_0011, 0b0000_0100]*24)
#print(data)
drv.spi_serial_write(1, data)

time.sleep(1e-6)
drv.gpio_pin_set(*PIC_PINS['UPDATE_TIA_CONF'])
time.sleep(1e-6)
drv.gpio_pin_reset(*PIC_PINS['UPDATE_TIA_CONF'])

In [123]:
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_OVERRIDE_EN'])

In [124]:
drv.gpio_pin_set(*PIC_PINS['READ_BIT'])
drv.gpio_pin_set(*PIC_PINS['READ_DPE'])

In [125]:
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE2'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE1'])
drv.gpio_pin_set(*PIC_PINS['NFORCE_SAFE0'])

In [126]:
drv.gpio_pin_set(*PIC_PINS['CONNECT_TIA'])

In [127]:
drv.gpio_pin_set(*PIC_PINS['CONNECT_COLUMN_T'])

In [128]:
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_PULSE'])

In [129]:
drv.gpio_pin_set(*PIC_PINS['DPE_EXT_SH'])

In [130]:
drv.gpio_pin_reset(*PIC_PINS['DPE_EXT_SH'])

In [131]:
drv.gpio_pin_reset(*PIC_PINS['DPE_EXT_PULSE'])

In [132]:
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<0>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<1>'])
drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_EN<2>'])
drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_EN<3>'])

data = drv.gpio_adc_read()
print(f'{data:013b}', end='\t')
print( f'{adc2volt(data):.3f} V' )

StrobeNum = 15
for strobe in range(StrobeNum):
    drv.gpio_pin_set(*PIC_PINS['ADC_FIFO_ADVANCE'])
    drv.gpio_pin_reset(*PIC_PINS['ADC_FIFO_ADVANCE'])
    data = drv.gpio_adc_read()
    print(f'{data:013b}', end='\t')
    #print(strobe)
    print( f'{adc2volt(data):.3f} V' )
print()

0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V

0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V

0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
0110000000010	0.507 V
01100000